## Nomenclatura dos Dados
* _p: Previous
    * _p _win_pct: Acertos prévios

In [16]:
import requests 
import json
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np
import itertools
import pickle
import os

In [17]:
def pickle_data(path, data):
    if not path.endswith('.pickle'):
        path += '.pickle'
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return

In [18]:
def read_pickle(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [31]:
def get_season(year):
    path = 'Data/{}/picks_{}.pickle'.format(year,year)
    return read_pickle(path)

In [32]:
def get_rank(year):
    path = 'Data/{}/rank_{}.csv'.format(year,year)
    return pd.read_csv(path)

In [19]:
def fetch_data(week, year=2019):
    url = "https://api.nflpickwatch.com/v1/nfl/expert-picks-su/" + str(year) + "/" + str(week)
    resp = requests.get(url).text
    data = json.loads(resp)
    return data

In [55]:
pd.json_normalize(data['expertPicks'])

,user_id,name,affiliation,url,rank,prev_rank,week_rank,t_season_id,t_week_id,t_wins,t_losses,t_ties,t_total,t_played,t_games,t_pick_pct,t_win_pct,t_loss_pct,t_tie_pct,t_wins_,t_losses_,t_ties_,t_total_,t_played_,t_games_,t_pick_pct_,t_win_pct_,t_loss_pct_,t_tie_pct_,t_c_season_id,t_c_week_id,t_c_wins,t_c_losses,t_c_ties,t_c_total,t_c_played,t_c_games,t_c_pick_pct,t_c_win_pct,t_c_loss_pct,t_c_tie_pct,t_c_wins_,t_c_losses_,t_c_ties_,t_c_total_,t_c_played_,t_c_games_,t_c_pick_pct_,t_c_win_pct_,t_c_loss_pct_,t_c_tie_pct_,l_season_id,l_week_id,l_wins,l_losses,l_ties,l_total,l_played,l_games,l_pick_pct,l_win_pct,l_loss_pct,l_tie_pct,l_wins_,l_losses_,l_ties_,l_total_,l_played_,l_games_,l_pick_pct_,l_win_pct_,l_loss_pct_,l_tie_pct_,l_c_season_id,l_c_week_id,l_c_wins,l_c_losses,l_c_ties,l_c_total,l_c_played,l_c_games,l_c_pick_pct,l_c_win_pct,l_c_loss_pct,l_c_tie_pct,l_c_wins_,l_c_losses_,l_c_ties_,l_c_total_,l_c_played_,l_c_games_,l_c_pick_pct_,l_c_win_pct_,l_c_loss_pct_,l_c_tie_pct_,p_wins,p_losses,p_ties,p_total,p_played,p_games,p_pick_pct,p_win_pct,p_loss_pct,p_tie_pct,p_c_wins,p_c_losses,p_c_ties,p_c_total,p_c_played,p_c_games,p_c_pick_pct,p_c_win_pct,p_c_loss_pct,p_c_tie_pct,picks.893.st,picks.893.s,picks.893.t,picks.893.r,picks.894.st,picks.894.s,picks.894.t,picks.894.r,picks.895.st,picks.895.s,picks.895.t,picks.895.r,picks.896.st,picks.896.s,picks.896.t,picks.896.r,picks.897.st,picks.897.s,picks.897.t,picks.897.r,picks.898.st,picks.898.s,picks.898.t,picks.898.r,picks.899.st,picks.899.s,picks.899.t,picks.899.r,picks.900.st,picks.900.s,picks.900.t,picks.900.r,picks.901.st,picks.901.s,picks.901.t,picks.901.r,picks.902.st,picks.902.s,picks.902.t,picks.902.r,picks.903.st,picks.903.s,picks.903.t,picks.903.r,picks.904.st,picks.904.s,picks.904.t,picks.904.r,picks.905.st,picks.905.s,picks.905.t,picks.905.r,picks.906.st,picks.906.s,picks.906.t,picks.906.r,picks.907.st,picks.907.s,picks.907.t,picks.907.r
0,260,Patrick Schmidt,Fansided,http://fansided.com/nfl/,1,24,5,2017,7,10,5,0,15,15,15,100.0,66.667,33.333,0,68,38,0,106,106,106,100.000,64.151,35.849,0,2017,7,1,3,0,4,4,4,100,25,75,0,24,13,0,37,37,37,100.000,64.865,35.135,0,2016,7.0,9,5,1,15,15,15,100.0,64.286,35.714,6.667,58,48,1,107,107,107,100.0,54.717,45.283,0.935,2016,7.0,4,1,1,6,6,6,100.0,80,20,16.667,18,20,1,39,39,39,100.0,47.368,52.632,2.564,165,99,2,266,266,267,99.625,62.500,37.500,0.752,39,42,1,82,82,83,98.795,48.148,51.852,1.220,1.0,896.0,KC,lose,1.0,897.0,BUF,win,1,898,CAR,lose,1.0,899.0,TEN,win,1,900,NO,win,1,901,JAX,win,1,902,AZ,lose,1.0,903.0,NYJ,lose,1,904,MIN,win,1.0,905.0,DAL,win,1.0,906.0,SEA,win,1,907,PIT,win,1.0,908.0,DEN,lose,1.0,909.0,NE,win,1.0,910.0,PHI,win
1,51,Jamey Eisenberg,CBS,http://www.cbssports.com/nfl/features/writers/...,2,10,4,2017,7,11,4,0,15,15,15,100.0,73.333,26.667,0,67,39,0,106,106,106,100.000,63.208,36.792,0,2017,7,2,2,0,4,4,4,100,50,50,0,21,16,0,37,37,37,100.000,56.757,43.243,0,2016,7.0,8,6,1,15,15,15,100.0,57.143,42.857,6.667,62,44,1,107,107,107,100.0,58.491,41.509,0.935,2016,7.0,3,2,1,6,6,6,100.0,60,40,16.667,20,18,1,39,39,39,100.0,52.632,47.368,2.564,174,91,2,267,267,267,100.000,65.660,34.340,0.749,44,38,1,83,83,83,100.000,53.659,46.341,1.205,1.0,896.0,KC,lose,1.0,897.0,BUF,win,1,898,CAR,lose,1.0,899.0,TEN,win,1,900,NO,win,1,901,JAX,win,1,902,AZ,lose,1.0,903.0,MIA,win,1,904,MIN,win,1.0,905.0,DAL,win,1.0,906.0,SEA,win,1,907,PIT,win,1.0,908.0,DEN,lose,1.0,909.0,NE,win,1.0,910.0,PHI,win
2,235,Jeff Ratcliffe,ProFootballFocus,https://twitter.com/PFF,2,17,4,2017,7,11,4,0,15,15,15,100.0,73.333,26.667,0,67,39,0,106,106,106,100.000,63.208,36.792,0,2017,7,2,2,0,4,4,4,100,50,50,0,25,12,0,37,37,37,100.000,67.568,32.432,0,2016,7.0,9,5,1,15,15,15,100.0,64.286,35.714,6.667,63,43,1,107,107,107,100.0,59.434,40.566,0.935,2016,7.0,3,2,1,6,6,6,100.0,60,40,16.667,21,17,1,39,39,39,100.0,55.263,44.737,2.564,170,95,2,267,267,267,100.000,64.151,35.849,0.749,39,43,1,83,83,83,100.000,47.561,52.439,1.205,1.0,896.0,KC,lose,1.0,897.0,BUF,win,1,898,CAR,lose,1.0,899.0,TEN,wi

In [73]:
def get_weekly_picks(week, data):
    games = pd.json_normalize(data['games'])[['id', 'rt_id', 'ht_id', 'game_state']]
    games.index = games.id
    cols = ['user_id']
    for g in list(games.id):
        cols.append("picks." + str(g) + ".t")
    picks = pd.json_normalize(data['expertPicks'])[cols].T
    picks.columns = picks.loc['user_id']
    picks = picks.drop(['user_id'])
    picks.index = games.index
    
    gp = pd.concat([games, picks], axis=1)
    gp = gp.fillna(0)

    gp = gp.rename(columns={"game_state":"game_result"})
    gp['game_result'] = np.where(gp['game_result']=='home-win', 1, 
                                np.where(gp['game_result']=='road-win', -1, 0))

    teams_dict = {}
    for ht in gp.ht_id:
        teams_dict[ht] = 1
    for rt in gp.rt_id:
        teams_dict[rt] = -1
    gp.iloc[:,4:] = gp.iloc[:,4:].replace(teams_dict)
    
    gp.insert(loc=4, column='week', value=int(data['expertPicks'][0]['t_week_id']))
    gp.insert(loc=4, column='year', value=int(data['expertPicks'][0]['t_season_id']))
    return gp

In [74]:
get_weekly_picks(17, data)

,id,rt_id,ht_id,game_result,year,week,260,51,235,272,31,32,266,262,277,182,168,251,236,34,27,110,270,44,254,106,275,70,112,103,157,37,48,160,269,174,90,146,17,124,136,189,78,205,67,23,271,59,154,75,261,76,156,255,216,284,131,111,107,88,188,280,121,213,234,217,268,283,18,52,95,161,214,137,63,130,186,267,49,282,278,80,185,248,190,259,22,204,114,65,285,61,96,101,228,265,281,264,181,152,172,184,256,191,66,276,98,195,263,129,279,292,29,19,238,196,295,297,127
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
893,893,KC,LV,1,2017,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,0,0,0
894,894,TB,BUF,1,2017,7,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,-1,-1,1,1,1,1,1,-1,-1,1,1,-1,1,1,1,1,-1,1,1,-1,1,-1,1,1,1
895,895,CAR,CHI,1,2017,7,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,1,1,-1,1,-1,1,-1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1
896,896,TEN,CLE,-1,2017,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0
897,897,NO,GB,-1,2017,7,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1
898,898,JAX,IND,-1,2017,7,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
899,899,AZ,LAR,1,2017,7,-1,-1,1,1,1,1,1,-1,1,1,-1,-1,1,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,-1,-1,1,1,-1,1,-1,-1,-1,1,1,1,1,1,-1,-1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,-1,1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,1,1,-1,1,1,1,1,-1,1,1,1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,1,1
900,900,NYJ,MIA,1,2017,7,-1,1,-1,-1,1,1,1,1,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,1,1,1,1,1,1,1,1,-1,-1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,1,1,1,-1,-1,1,-1,1,1,1,1,1,1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,1,1,1,-1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,-1,1,1,-1,1,-1,-1,0,0,0
901,901,BAL,MIN,1,2017,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,-1,-1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,1,-1,1,1,1


In [59]:
games = pd.json_normalize(data['games'])[['id', 'rt_id', 'ht_id', 'game_state']]
games.index = games.id
cols = ['user_id', 't_season_id', 't_week_id']
for g in list(games.id):
    cols.append("picks." + str(g) + ".t")
picks = pd.json_normalize(data['expertPicks'])[cols].T

In [72]:
data['expertPicks'][0]['t_season_id']['t_week_id']

,id,rt_id,ht_id,game_result,121,47,97,146,98,125,42,61,114,32,143,33,110,145,29,149,57,45,127,88,159,63,27,112,113,19,128,66,137,89,90,92,37,38,21,160,136,106,41,18,117,120,46,26,54,55,157,60,64,166,85,154,144,70,138,141,76,78,79,34,35,142,132,17,23,24,30,39,40,43,44,48,49,51,56,59,65,67,68,72,80,82,93,94,95,96,101,107,111,115,116,122,123,130,16,133,140,147,150,153,161,162,163,167,27068,134,102,87,69,148,36,165,118,152,20,108,105,103,129,28,139,73,71,135,31,22,75,119,131,58,124,83,164,25,52
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,GB,SEA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,-1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,-1,1,1
2,2,NO,ATL,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,3,MIN,LAR,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,-1,-1,1,1,-1,1,-1,-1,1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,-1,-1,1,-1,1,1,-1,1,1,1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,0,1,1,1,1,1,1,1,1,-1,1,1
4,4,CLE,PIT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,5,JAX,PHI,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1
6,6,LV,NYJ,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,-1,-1,1,1,1,1,1,-1,1,-1,1,1,1,1,1,-1,1,1,-1,1,-1,1,1,1,-1,1,1,-1,1,1
7,7,CIN,BAL,-1,-1,1,-1,-1,-1,1,-1,-1,1,1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,1,1,-1,1,-1,-1,-1,-1,-1,1,-1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,-1,1,-1,1,1,-1,1,1,1,-1,1,1,1,-1,1,-1,1,-1,1,-1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1
8,8,BUF,CHI,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,9,WAS,HOU,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,-1,-1,1,1,-1,-1,1,1,-1,1,-1,-1,-1,1,1,-1,1,1,-1,1,1,1,1,-1,-1,-1,1,1,1,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,1,-1,1,1,-1,-1,1,1,-1,1,-1,-1,1,-1,1,-1,-1,1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,1,-1,1,-1,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,1,1,-1,-1


,user_id,blo,bla,name,affiliation,prev_rank,p_wins,p_win_pct,season_hits,week0_rank,week1_hits,week1_rate,week1_pct,week1_rank,week2_hits,week2_rate,week2_pct,week2_rank,week3_hits,week3_rate,week3_pct,week3_rank,week4_hits,week4_rate,week4_pct,week4_rank,week5_hits,week5_rate,week5_pct,week5_rank,week6_hits,week6_rate,week6_pct,week6_rank,week7_hits,week7_rate,week7_pct,week7_rank,week8_hits,week8_rate,week8_pct,week8_rank,week9_hits,week9_rate,week9_pct,week9_rank,week10_hits,week10_rate,week10_pct,week10_rank,week11_hits,week11_rate,week11_pct,week11_rank,week12_hits,week12_rate,week12_pct,week12_rank,week13_hits,week13_rate,week13_pct,week13_rank,week14_hits,week14_rate,week14_pct,week14_rank,week15_hits,week15_rate,week15_pct,week15_rank,week16_hits,week16_rate,week16_pct,week16_rank,week17_hits,week17_rate,week17_pct,week17_rank,week18_hits,week18_rate,week18_pct,week18_rank,week19_hits,week19_rate,week19_pct,week19_rank,week20_hits,week20_rate,week20_pct,week20_rank,week21_hits,week21_rate,week21_pct,week21_rank
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
51,51,2,3,Jamey Eisenberg,CBS,1,180,67.925,0,1.0,9,0,0.5625,0,12,0,0.7500,0,9,0,0.5625,0,7,0,0.466667,0,7,0,0.466667,0,5,0,0.357143,0,9,0,0.642857,0,13,0,0.866667,0,6,0,0.428571,0,5,0,0.384615,0,11,0,0.785714,0,10,0,0.714286,0,10,0,0.6250,0,12,0,0.7500,0,10,0,0.6250,0,11,0,0.6875,0,11,0,0.6875,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106,106,2,3,Pete Prisco,CBS,2,179,67.547,0,2.0,9,0,0.5625,0,10,0,0.6250,0,10,0,0.6250,0,8,0,0.533333,0,6,0,0.400000,0,4,0,0.285714,0,9,0,0.642857,0,13,0,0.866667,0,8,0,0.571429,0,7,0,0.538462,0,9,0,0.642857,0,9,0,0.642857,0,9,0,0.5625,0,11,0,0.6875,0,10,0,0.6250,0,9,0,0.5625,0,11,0,0.6875,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27064,27064,2,3,Mitch Goldich,MMQB,3,178,67.170,0,3.0,10,0,0.6250,0,11,0,0.6875,0,11,0,0.6875,0,7,0,0.466667,0,10,0,0.666667,0,9,0,0.642857,0,10,0,0.714286,0,13,0,0.866667,0,9,0,0.642857,0,3,0,0.230769,0,10,0,0.714286,0,9,0,0.642857,0,8,0,0.5000,0,12,0,0.7500,0,11,0,0.6875,0,10,0,0.6250,0,11,0,0.6875,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
76,76,2,3,Nathan Jahnke,ProFootballFocus,4,177,66.792,0,6.0,11,0,0.6875,0,11,0,0.6875,0,10,0,0.6250,0,6,0,0.400000,0,8,0,0.533333,0,9,0,0.642857,0,9,0,0.642857,0,13,0,0.866667,0,8,0,0.571429,0,3,0,0.230769,0,14,0,1.000000,0,13,0,0.928571,0,11,0,0.6875,0,12,0,0.7500,0,10,0,0.6250,0,11,0,0.6875,0,10,0,0.6250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,98,2,3,Dave Richard,CBS,4,177,66.792,0,6.0,9,0,0.5625,0,11,0,0.6875,0,12,0,0.7500,0,6,0,0.400000,0,7,0,0.466667,0,5,0,0.357143,0,8,0,0.571429,0,12,0,0.800000,0,7,0,0.500000,0,3,0,0.230769,0,9,0,0.642857,0,11,0,0.785714,0,7,0,0.4375,0,12,0,0.7500,0,10,0,0.6250,0,11,0,0.6875,0,11,0,0.6875,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53447,53447,2,3,Damon Marx,DallasMorningNews,56,0,0.000,0,111.0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,13,0,0.866667,0,8,0,0.571429,0,5,0,0.384615,0,12,0,0.857143,0,10,0,0.714286,0,6,0,0.3750,0,9,0,0.5625,0,12,0,0.7500,0,10,0,0.6250,0,8,0,0.5000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44212,44212,2,3,Cam Mellor,ProFootballFocus,56,0,0.000,0,111.0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,13,0,0.866667,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
166,166,2,3,Rod Woodson,Westwood One,56,0,0.000,0,111.0,12,0,0.7500,0,9,0,0.5625,0,10,0,0.6250,0,7,0,0.466667,0,9,0,0.600000,0,7,0,0.500000,0,8,0,0.5714

In [21]:
tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_diff'])

In [22]:
def get_votes(week, rank, picks, top, strategy="rank_squared", tight=tg):
    best = rank.sort_values(by='week' + str(week-1) + '_rank')
    best = best.iloc[0:top]
#     best['votes'] = (top - best['week' + str(week-1) + '_rank'])
    
    if strategy == "rank_squared":
        best['pct_rank'] = best['week' + str(week-1) + '_rank'].rank(pct=True, ascending=False)
        best['votes'] = best.pct_rank**2
        best['votes'] = np.where(best['votes'] < 0, 0, best['votes'])
        
    elif strategy == 'democracy':
        best['votes'] = 1
    
    elif strategy == 'n-rank':
        best['votes'] = best['votes'] = (top - best['week' + str(week-1) + '_rank'] + top/2)/(top/2)
        
    
    bid = list(best.user_id)
    
    votes = picks[bid]*best['votes']
    votes['id'] = picks.id
    votes['rt_id'] = picks['rt_id']
    votes['ht_id'] = picks['ht_id']
    votes['game_result'] = picks['game_result']
    
    votes['abs_vote'] = best['votes'].sum()
    votes['vote'] = votes[bid].sum(axis=1)
    
    votes['vote_ratio'] = votes.vote.abs()/votes.abs_vote
    
    temp = votes[['id', 'rt_id','ht_id','game_result','vote','abs_vote', 'vote_ratio']]
    temp = temp[temp.vote_ratio <= 0.1]
    temp['vote_first'] = best.iloc[0].votes

    tight = pd.concat([tight, temp])
        
    
    votes['vote'] = np.where(votes['vote']==0, votes[best.iloc[0].user_id], votes['vote']) #In case of votes summing up to zero, the first ranked expert gets the Minerva vote.
    
    votes['bet'] = np.sign(votes.vote)
    return votes, tight

In [8]:
year = 2019
bla = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
bla.index = bla.user_id

In [23]:
def get_rank(year):
    data = fetch_data(year=year, week=1)
    rank = pd.json_normalize(data['expertPicks'])[['user_id', 'name', 'affiliation', 'prev_rank', 'p_wins', 'p_win_pct']]
    rank['season_hits'] = 0
    rank['week0_rank'] = rank.prev_rank.rank(method='max')
    ht = ['week' + str(w) + '_hits' for w in range (1,22)]
    rt = ['week' + str(w) + '_rate' for w in range (1,22)]
    pc = ['week' + str(w) + '_pct' for w in range (1,22)]
    rk = ['week' + str(w) + '_rank' for w in range (1,22)]

    for w in range (0,21):
        rank[ht[w]] = 0
        rank[rt[w]] = 0
        rank[pc[w]] = 0
        rank[rk[w]] = 0

    rank.index = rank.user_id
    
    for week in range (1, 18):
        data = fetch_data(week, year)
        week_stats = pd.json_normalize(data['expertPicks'])[['user_id', 't_wins', 't_games']]
        week_stats.index = week_stats.user_id
        rank['week'+str(week)+'_hits']=week_stats.t_wins
        rank['week'+str(week)+'_pct']=week_stats.t_wins/week_stats.t_games
#         for d in data['expertPicks']:
#             if d['picks'] == {} and d['user_id'] in rank.user_id:
#                 rank = rank.drop(d['user_id'])
#                 print("Removed analyst " + str(d['user_id']))
        

    return rank

In [24]:
def calculate_rank(rank, week, hist_modifier, season_hits_modifier, hist_decay, last_season_bonus):
#     for week in range(1, week+1):
    rate = 'week' + str(week) + "_rate"
    rank[rate] = 0
    rank['season_hits'] = 0
    for w in range (1, week+1):
        rank['season_hits'] += rank['week'+str(w)+'_hits']

    if hist_decay:
        hd = {}
        width = max(0,(week-hist_modifier))
        rg = min(hist_modifier, week)
        for i in range(rg):
            hd[week-rg+i+1] = (i+1)/(week-width)

    n = week
    while (n > week - hist_modifier) and (n > 0):
        if hist_decay:
            rank[rate] += rank['week' + str(n) + "_hits"]*hd[n]
#                     print("Week: {}; n: {}; Decay: {}".format(week, n, hd[n]))

        else:
            rank[rate] += rank['week' + str(n) + "_hits"]   
        n -= 1

    bonus_weeks = 2
    rank[rate] += rank['p_wins']/16 * max(bonus_weeks-week,0) * last_season_bonus
    rank[rate] = rank[rate]/(min(hist_modifier, week))

    rank[rate] += (rank['season_hits'] * season_hits_modifier)/week
    rank['week' + str(week) + "_rank"] = rank[rate].rank(method='max', ascending=False)

    return rank 

In [25]:
def win_a_bet(hp):
    tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_ratio', 'vote_first'])
    results_cols = []
    hits_cols=[]
    gms_cols = []
    for y in range(2014,2020):
        results_cols+= [str(y)+"_hits", str(y)+"_gms", str(y)+"_rate"]
        hits_cols.append(str(y)+'_hits')
        gms_cols.append(str(y)+'_gms')

    results = pd.DataFrame(columns=results_cols)

    for year in range (2014,2020):
        season = read_pickle('Data/' + str(year) + '/' + "picks_" + str(year) + ".pickle")
        total_hits = 0
        total_games = 0
        for week in range(1,18):
            rank = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
            rank.index = rank.user_id
            rank = calculate_rank(rank, hist_modifier=hp['hist_modifier'], season_hits_modifier=hp['season_hits_modifier'], hist_decay=hp['hist_decay'], last_season_bonus=hp['last_season_bonus'])

    #         season[week] = get_weekly_picks(week, data)
            picks = season[week]

            votes, tg = get_votes(week, rank, picks, top=hp['n_top_experts'], strategy=hp['vote_strategy'], tight=tg)
            picks['vote'] = votes.vote
            picks['bet'] = votes.bet
            picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)

            hits = picks['hit'].sum()
            games = picks['hit'].count()
            total_hits += hits
            total_games += games

            results.loc['week'+str(week), str(year)+'_gms'] = games
            results.loc['week'+str(week), str(year)+'_hits'] = hits
            results.loc['week'+str(week), str(year)+'_rate'] = hits/games

        results.loc['total', str(year)+'_gms'] = total_games
        results.loc['total', str(year)+'_hits'] = total_hits
        results.loc['total', str(year)+'_rate'] = total_hits/total_games
        print (tg.shape)

        print("{} Hits - Total: {}/{} = {:.2%}".format(year,total_hits,total_games,total_hits/total_games))

    results['mean_hits'] = results[hits_cols].mean(1)
    results['total_hits'] = results[hits_cols].sum(1)
    results['mean_gms'] = results[gms_cols].mean(1)
    results['total_gms'] = results[gms_cols].sum(1)
    results['total_rate'] = results['total_hits']/results['total_gms']
    #         print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))

    return results, tg

In [ ]:
season = {}
total_hits = 0
total_games = 0
for week in range(1,18):
    data = fetch_data(week)
    season[week] = get_weekly_picks(week, data)
    picks = season[week]
    votes = get_votes(week, rank, picks, top=_TOP_N)
    picks['vote'] = votes.vote
    picks['bet'] = votes.bet
    picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)
    hits = picks['hit'].sum()
    games = picks['hit'].count()
    total_hits += hits
    total_games += games
    print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))
print("Hits - TOTAL: {}/{} = {:.2%}".format(total_hits,total_games,total_hits/total_games))

In [ ]:
rank=get_rank()

In [75]:
def gather_data(year):
    season = {}
    for week in range(1,22):
        data = fetch_data(week, year)
        season[week] = get_weekly_picks(week, data)
        
    if not os.path.exists('Data/' + str(year)):
        os.makedirs('Data/' + str(year))
        
    pickle_data(path='Data/' + str(year) + '/picks_' + str(year) + '.pickle', data=season)     
    
    return

In [78]:
for y in range(2014,2020):
    print(y)
    gather_data(y)

2014
2015
2016
2017
2018
2019


In [204]:
for year in range(2014,2020):
    r = get_rank(year)
#     r = calculate_rank(r, hist_modifier=hyperparameters['hist_modifier'], season_hits_modifier=hyperparameters['season_hits_modifier'], consider_last_season=hyperparameters['consider_last_season'])
    r.to_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv', index=False)

In [ ]:
%%time
for year in range(2014, 2016):
    print(year)
    gather_data(year)

In [27]:
hyperparameters = {
    'hist_modifier': 4,
    'n_top_experts': 18,
    'last_season_bonus': 0.5,
    'season_hits_modifier': 0,
    'vote_strategy': 'n-rank',
    'hist_decay': True,
}

In [88]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

In [98]:
full_seasons = []
for y in range(2014, 2020):
    season_dict = get_season(y)
    season_concat = [season_dict[w] for w in season_dict]
    full_seasons.append(pd.concat(season_concat))
    
df = pd.concat(full_seasons)
df = df[df.game_result!=0]

In [99]:
split = int(df.shape[0] * 0.8)
df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)


model = RandomForestClassifier(n_estimators=200, 
                               bootstrap = False,
                               max_features = 'auto',
                               max_depth = 12)

model.fit(df_train.iloc[:,4:], df_train.game_result)

RandomForestClassifier(bootstrap=False, max_depth=12, n_estimators=200)

In [100]:
df_test['pred'] = model.predict(df_test.iloc[:,4:])
df_test['hit'] = np.where(df_test.pred == df_test.game_result, 1, 0)
df_test.hit.sum()/df_test.hit.count()

0.628125

In [104]:
df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

RSEED = 42
# train_labels = 
param_grid = {
    'n_estimators': np.linspace(10, 400, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 40, 100).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [1,2,3,4,5,8,10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = RSEED)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = 10, 
                        scoring = 'accuracy', cv = 3,                                                                  
                        
                        n_iter = 1500, verbose = 1, random_state=RSEED, return_train_score=True)

# Fit 
rs.fit(df_train.iloc[:,4:], df_train.game_result)

Fitting 3 folds for each of 1500 candidates, totalling 4500 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.6s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   43.4s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  3.5min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed:  5.5min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed:  8.1min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed: 11.1min
[Parallel(n_jobs=10)]: Done 3180 tasks      | elapsed: 14.5min
[Parallel(n_jobs=10)]: Done 4030 tasks      | elapsed: 18.5min
[Parallel(n_jobs=10)]: Done 4500 out of 4500 | elapsed: 20.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=1500, n_jobs=10,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 3, 3, 3, 4, 4, 4, 5,
                                                      5, 5, 6, 6, 7, 7, 7, 8, 8,
                                                      8, 9, 9, 10, 10, 10, 11,
                                                      11, 11, 12, 12, 13, 13, ...],
                                        'max_features': ['auto', 'sqrt', None,
                                                         0.5, 0.6, 0.7,
                                                         0.7999999999999999,
                                                         0.8999999999999999],
                                        'max_leaf_nodes': [None, 10, 10, 10...
       264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288,
       290, 292, 294, 296, 298,

In [105]:
best_model = rs.best_estimator_

df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

# df_test['pred'] = cross_val_predict(best_model, df_test.iloc[:,4:], df_test.game_result, cv=5)
# df_train['pred'] = cross_val_predict(best_model, df_train.iloc[:,4:], df_train.game_result, cv=5)

df_test['pred'] = best_model.predict(df_test.iloc[:,4:])
df_train['pred'] = best_model.predict(df_train.iloc[:,4:])

df_test['hit'] = np.where(df_test.game_result==df_test.pred, 1, 0)
df_train['hit'] = np.where(df_train.game_result==df_train.pred, 1, 0)

print(rs.best_params_)
print(df_test['hit'].sum()/df_test['hit'].count())
print(df_train['hit'].sum()/df_train['hit'].count())

{'n_estimators': 274, 'min_samples_split': 5, 'max_leaf_nodes': 22, 'max_features': 'auto', 'max_depth': 4, 'bootstrap': False}
0.63125
0.7178683385579937
